In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
def preprocess_dataset(folderpath, dest_path, limit:int=None):
  folder_files = os.listdir(folderpath)

  for i, file in enumerate(folder_files, start=1):
    base_image = cv2.imread(f'{folderpath}/{file}')
    base_image = cv2.resize(base_image, (224,224))

    enhanced_image = enhance_contrast(base_image, False)
    max_pixel_diff, enhanced_green, g = enhance_green_channel(image=enhanced_image,
                                        show_channels=False,
                                        show_enhanced=False)

    blood_vessel_mask = extract_blood_vessels(image=enhanced_green,
                                            show_extracted=False)

    # Inside preprocess_dataset function, when calling extract_outliers
    outlier_mask = extract_outliers(image=g, g=g, max_pixel_diff=max_pixel_diff, show_extracted=False)


    overlap_mask = cv2.bitwise_and(blood_vessel_mask, outlier_mask)
    outlier_mask = np.subtract(outlier_mask, overlap_mask)
    outlier_mask[outlier_mask == 50] = 225

    merged_mask = cv2.bitwise_or(blood_vessel_mask, outlier_mask)
    merged_mask[merged_mask == 0] = 100
    
    rgb_mask = cv2.cvtColor(merged_mask,cv2.COLOR_GRAY2RGB)
    result = np.hstack((base_image, rgb_mask))
    cv2.imshow("output", result)

    final_output = cv2.resize(rgb_mask, (512,512))
    cv2.imwrite(f"{dest_path}/{file}", final_output) 

    if limit == None:
        pass

    elif i >= limit:
        break

def enhance_contrast(image:np.ndarray, show_enchanced:bool):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB) #convert rgb image to lab channels
    L, a, b = cv2.split(lab) #split lab channels

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) #create Contrast Limited Adaptive Histogram Equalization (CLAHE)
    cL = clahe.apply(L) #apply to lab channel to increase contrast

    new_img = cv2.merge((cL,a,b)) #create new lab enhanced image
    enhanced_img = cv2.cvtColor(new_img, cv2.COLOR_LAB2BGR) #convert from lab to rgb

    if show_enchanced:
      result = np.hstack((image, enhanced_img))
      cv2.imshow("enhanced image", result) #update to local version @weixuan

    return enhanced_img


def enhance_green_channel(image:np.ndarray, kernel_size:tuple=(75,75), show_channels:bool=False, show_enhanced:bool=False):
    b,g,r = cv2.split(image)
    if show_channels:
      cv2.imshow("blue channel", b)
      cv2.imshow("green channel", g)
      cv2.imshow("red channel", r)

    kernel = np.ones(kernel_size, np.uint8)
    opening = cv2.morphologyEx(g, cv2.MORPH_OPEN, kernel)
    morph_green = cv2.subtract(g, opening)

    max_pixel_diff = np.max(morph_green)

    min_intensity = np.min(morph_green)
    max_intensity = np.max(morph_green)

    enhanced_green = ((morph_green - min_intensity) / (max_intensity - min_intensity)) * 255
    enhanced_green = enhanced_green.astype(np.uint8)

    if show_enhanced:
      result = np.hstack((g, enhanced_green))
      cv2.imshow("enhanced green channel", result)

    return max_pixel_diff, enhanced_green, g


def extract_blood_vessels(image:np.ndarray, block_size:int=31, calculated_mean:int=21, area_limit:int=3, show_extracted:bool=False):
    if block_size % 2 ==0:
      raise Exception("block_size must be an odd integer")

    while True:
      adaptive_threshold = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, calculated_mean)
      contours, _ = cv2.findContours(adaptive_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      contours = sorted(contours, key=cv2.contourArea, reverse=True)
      contours = [cnt for cnt in contours if cv2.contourArea(cnt) > area_limit]
      max_contour_area = max([cv2.contourArea(cnt) for cnt in contours])
      if max_contour_area >= 10000:
        block_size += 2
        calculated_mean += 3
      
      else:
        break

    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)

    gray_mask = np.copy(mask)
    gray_mask[mask == 255] = 50

    if show_extracted:
      result = np.hstack((image, gray_mask))
      cv2.imshow("vein extraction", result)

    return gray_mask


def extract_outliers(image: np.ndarray, g: np.ndarray, max_pixel_diff: int = 125, show_extracted: bool = False):
    # Initialize variables to None
    binary = binary2 = binary3 = None

    # Your existing conditional logic here...
    if max_pixel_diff < 165:
        _, binary = cv2.threshold(image, max_pixel_diff, 255, cv2.THRESH_BINARY)
        _, binary2 = cv2.threshold(image, max_pixel_diff, 255, cv2.THRESH_BINARY)
        _, binary3 = cv2.threshold(image, max_pixel_diff * 1.1, 255, cv2.THRESH_BINARY)

    elif 165<= max_pixel_diff <200:
      _, binary = cv2.threshold(image, max_pixel_diff*0.8, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.85, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff*1.1, 255, cv2.THRESH_BINARY)

    elif 200<= max_pixel_diff <215:
      _, binary = cv2.threshold(image, max_pixel_diff*0.7, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.75, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff*0.8, 255, cv2.THRESH_BINARY)

    elif 215 <= max_pixel_diff < 230:
      _, binary = cv2.threshold(image, max_pixel_diff*0.75, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.85, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff*0.9, 255, cv2.THRESH_BINARY)

    elif 230<= max_pixel_diff <245:
      _, binary = cv2.threshold(image, max_pixel_diff*0.45, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.5, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff, 255, cv2.THRESH_BINARY)

    elif max_pixel_diff >245:
      _, binary = cv2.threshold(image, max_pixel_diff*0.7, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.75, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff, 255, cv2.THRESH_BINARY)



    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours2, _ = cv2.findContours(binary2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours3, _ = cv2.findContours(binary3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(g)
    mask2 = np.zeros_like(g)
    mask3 = np.zeros_like(g)

    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
    cv2.drawContours(mask2, contours2, -1, (255), thickness=cv2.FILLED)
    cv2.drawContours(mask3, contours3, -1, (255), thickness=cv2.FILLED)

    merge_mask = np.subtract(mask, mask2)
    merge_mask = np.add(merge_mask, mask3)

    gray_mask = np.copy(merge_mask)
    gray_mask[mask == 255] = 50

    if show_extracted:
      result = np.hstack((image, gray_mask))
      cv2.imshow("outlier extraction", result)

    return gray_mask


In [ ]:
preprocess_dataset(r'E:\Diabetic_Retinopathy_Detection\input\grading_images\processed_images', r'E:\Diabetic_Retinopathy_Detection\input\grading_images\augment')


In [ ]:
import os
import cv2
import numpy as np

folder_path = r'E:\Diabetic_Retinopathy_Detection\input\grading_images\img'
processed_folder_path = r'E:\Diabetic_Retinopathy_Detection\input\grading_images\processed_images'

# Ensure the processed images directory exists
if not os.path.exists(processed_folder_path):
    os.makedirs(processed_folder_path)
    print(f"Created directory: {processed_folder_path}")

# Define a list of acceptable file extensions
acceptable_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']

# List files in the source folder and filter by acceptable file extensions
all_files = [f for f in os.listdir(folder_path) 
             if os.path.splitext(f)[1].lower() in acceptable_extensions]

if not all_files:
    print("No image files found in the folder.")
else:
    print(f"Found {len(all_files)} image files.")

selected_files = np.random.choice(all_files, size=(len(all_files)), replace=False)

for file_name in selected_files:
    file_path = os.path.join(folder_path, file_name)
    processed_file_path = os.path.join(processed_folder_path, file_name)
    sample_img = cv2.imread(file_path)
    
    # Check if the image was loaded properly
    if sample_img is None:
        print(f"Failed to load image: {file_name}")
        continue
    
    # Convert to RGB and apply processing
    sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
    ben_sample_img = cv2.addWeighted(sample_img, 4, cv2.GaussianBlur(sample_img, (0,0), 10), -4, 128)
    ben_sample_img_bgr = cv2.cvtColor(ben_sample_img, cv2.COLOR_RGB2BGR)
    
    # Save the processed image
    cv2.imwrite(processed_file_path, ben_sample_img_bgr)
    print(f"Processed and saved: {file_name}")

print("Processing complete.")
